# Tensorflow Functions

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
def cube(x):
    return x**3

In [3]:
cube(3)

27

In [4]:
cube(tf.constant(2.))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [ ]:
# tf.function is used to convert python function to tensorflow fun
tf_cube = tf.function(cube)
tf_cube

In [ ]:
# Now we can use this func exactly like python func
tf_cube(2)

In [ ]:
tf_cube(tf.constant(2.))

In [ ]:
# The more convinent way is to create decorator
@tf.function
def tf_cube(x):
    return x**3

In [ ]:
# We can get python version of tf function by
tf_cube.python_function(2)

### 1. TF Functions and Concreate Functions
Tensorflow analysis the source code of python function to capture all control flow statements, like for loops, break, if-statement. This step is call `AutoGraph`. The reason tensorflow to analyse the source code is that python doesn't provide any other way to capture control flow of statements. It offers magic function like __add__(), __mul__(),for operator like +, * but don't for Forloop while loop, if-statement. After analyse, `AutoGraph` outputs an upgraded version of that function in tensorflow operations, such as for-loop replaced with appropriate tf.while_loop() if statement with tf.cond()

In [ ]:
dir2 = lambda x: [i for i in dir(x) if i[0] != '_'] # Some global functions
dir2(tf_cube)

In [ ]:
# Concrete func are those which created by AutoGraph
concreate_function = tf_cube.get_concrete_function(tf.constant(2.))
concreate_function

In [ ]:
concreate_function(tf.constant(2.))

In [ ]:
concreate_function is tf_cube(tf.constant(2.))

In [ ]:
concreate_function is tf_cube.get_concrete_function(tf.constant(2.))

### 2. Exploring Function Definitions and Graphs

In [ ]:
# This if generated functions source code
print(tf.autograph.to_code(cube))

In [ ]:
dir2(concreate_function)

In [ ]:
# Return all operations that Autograph generated
ops = concreate_function.graph.get_operations()
ops

In [ ]:
ops[2].inputs

In [ ]:
ops[2].outputs

In [ ]:
concreate_function.graph.get_operation_by_name('pow/y')

In [ ]:
concreate_function.graph.get_operation_by_name('Identity')

In [ ]:
concreate_function.graph.get_tensor_by_name('Identity:0')

In [ ]:
concreate_function.function_def.signature

### 3. A closer look at Tracing: How TF functions Trace python functions to extract their computation graphs

In [ ]:
# Lets define cube function again
@tf.function
def tf_cube(x):
    # Printing what we passing to this function
    print("x: ", x)
    return x**3

In [ ]:
# Initial trace
# we can see x is symnbolic tensor: it has name, shape, dtype but no value 
result = tf_cube(tf.constant(2.))
result

In [ ]:
# If we paas tensorflow data resource to the function
# then it will create the graph with name, shape, dtype
# if we recall the function with same dtype and shape it will not go into tracing

# You can see print() statement doesn't execute bcz it is not 
# go into graph bcz it is not tensorflow function
tf_cube(tf.constant(3.))

In [ ]:
tf_cube(2) # new python value: trace

In [ ]:
tf_cube(3) # new python value: trace

In [ ]:
tf_cube(tf.constant([[2., 2.]])) # New shape: trace 

In [ ]:
# It means if we do any python logic it will not run in second time

tf_cube(tf.constant([[4., 5.]])) # Same shape: no Tracing

In [ ]:
# If you want to restrict to the input shape
# Like in image analysis
@tf.function(input_signature=[tf.TensorSpec([None, 28, 28], tf.float32)])
def shrink(img):
    print("Tracing: ", img)
    return tf.shape(img[:,::2,::2]) # drop alternate rows and columns


In [ ]:
b1 = tf.random.uniform([10,28,28])
b2 = tf.random.uniform([20,28,28])
b3 = tf.random.uniform([1,4,4])
imgs = shrink(b1) # Tracing
imgs

In [ ]:
imgs = shrink(b2) # No Trace: graph already build with same shape, dtype
imgs

In [ ]:
# You can see we defined the input_signature
try:
    imgs = shrink(b3) # Through error
    print(imgs)
except ValueError as e:
    print(e)

### 4. Using AutoGraph to Capture Control Flow

In [ ]:
# for loop is work as an static loop when we convert python function to tensorflow function
@tf.function
def add(x):
    for i in range(5):
        x+=1
    return x

In [ ]:
add(tf.constant(5))

In [ ]:
# We can see for executed 5 time. tf unrool the loop when graph is created
add.get_concrete_function(tf.constant(5)).graph.get_operations()

In [ ]:
# We can avoid this by using tf.while_loop()
# This is the best technique to understand how loops work (function calling function)
@tf.function
def add(x):
    condition = lambda i, x: tf.less(i, 10)
    body      = lambda i, x: (tf.add(i, 1), tf.add(x, 1))
    final_i, final_x = tf.while_loop(condition, body, [tf.constant(0), x])
    return final_x

In [ ]:
add(tf.constant(5))


In [ ]:
add.get_concrete_function(tf.constant(5)).graph.get_operations()

In [ ]:
# Instead we can use tf.range() that captured by AutoGraph
@tf.function
def add(x):
    for i in tf.range(10):
        x += 1
    return x

In [ ]:
add(tf.constant(5))

In [ ]:
add.get_concrete_function(tf.constant(5)).graph.get_operations()

### 5. Handling Variables and Other Resources in TF Functions

In [ ]:
# When we pass resources to the TF Functions it is gets passed by reference
counter = tf.Variable(0.)

@tf.function
def increment(counter, c=1):
    return counter.assign_add(c)

increment(counter) # counter=1
increment(counter) # counter=2
counter

In [ ]:
function_def = increment.get_concrete_function(counter).function_def
function_def.signature

In [ ]:
function_def.signature.input_arg[0]

In [ ]:
# It will also work with global variable
counter = tf.Variable(0.)

@tf.function
def increment(c=1): # didn't pass in function
    return counter.assign_add(c)

increment()
increment()
counter

In [ ]:
function_def = increment.get_concrete_function().function_def
function_def.signature

In [ ]:
function_def.signature.input_arg[0]

In [ ]:
# Using global variable lead to messy. Instead define variables in class
class Counter:
    def __init__(self):
        self.counter = tf.Variable(0)

    @tf.function
    def increment(self, c=1):
        return self.counter.assign_add(c)

In [ ]:
c = Counter()
c.increment()
c.increment()

In [ ]:
@tf.function
def add(x):
    for i in tf.range(10):
        x += 1
    return x

print(tf.autograph.to_code(add.python_function))

In [ ]:
#shows how to display the autograph code with syntax highlighting
def display_tf_code(func):
    from IPython.display import display, Markdown
    if hasattr(func, "python_function"):
        func = func.python_function
    code = tf.autograph.to_code(func)
    display(Markdown(f'```python\n{code}\n```'))

In [ ]:
display_tf_code(add)

### 6. Using TF Functions with tf.keras or not

By default, `tf.keras` will automatically convert your custom code into TF Functions, no need to use `tf.function():`

In [ ]:
house = fetch_california_housing()
x_train, x_test, y_train, y_test   = train_test_split(house.data, house.target, test_size=0.1, random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

scaler, scaler2 = StandardScaler(), StandardScaler()
x_train_s = scaler.fit_transform(x_train)
x_test_s  = scaler.transform(x_test)
x_valid_s = scaler.transform(x_valid)

print("Independent variables : ", x_train_s.shape, x_valid_s.shape, x_test_s.shape)
print("Target variables      : ", y_train.shape, y_valid.shape, y_test.shape)

In [ ]:
# Custom loss function
def my_mse(y_true, y_pred):
    print("Tracing loss my_mse()")
    return tf.reduce_mean(tf.square(y_pred - y_true))


# Custom metric function
def my_mae(y_true, y_pred):
    print("Tracing metric my_mae()")
    return tf.reduce_mean(tf.abs(y_pred - y_true))


# Custom layer
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel', shape=(input_shape[1], self.units),initializer='uniform', trainable=True)
        self.biases = self.add_weight(name='bias', shape=(self.units,),initializer='zeros',trainable=True)
        super().build(input_shape)

    def call(self, X):
        print("Tracing MyDense.call()")
        return self.activation(X @ self.kernel + self.biases)

# Custom model
class MyModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = MyDense(30, activation="relu")
        self.hidden2 = MyDense(30, activation="relu")
        self.output_ = MyDense(1)

    def call(self, input):
        print("Tracing MyModel.call()")
        hidden1 = self.hidden1(input)
        hidden2 = self.hidden2(hidden1)
        concat = tf.keras.layers.concatenate([input, hidden2])
        output = self.output_(concat)
        return output

In [ ]:
model = MyModel()
model.compile(loss=my_mse, optimizer="adam", metrics=[my_mae])
model.fit(x_train_s, y_train, epochs=2,validation_data=(x_valid_s, y_valid))
model.evaluate(x_test_s, y_test)

We can turn this off by creating the model with `dynamic=True` (or calling `super().__init__(dynamic=True, **kwargs)` in the model's constructor):

In [ ]:
# NOw the custom function called
model = MyModel(dynamic=True)
model.compile(loss=my_mse, optimizer="adam", metrics=[my_mae])
model.fit(x_train_s[:64], y_train[:64], epochs=1,validation_data=(x_valid_s[:64], y_valid[:64]), verbose=0)
model.evaluate(x_test_s[:64], y_test[:64], verbose=0)

Alternatively, we can compile a model with `run_eagerly=True`

In [ ]:
model = MyModel(dynamic=True)
model.compile(loss=my_mse, optimizer="adam", metrics=[my_mae], run_eagerly=True)
model.fit(x_train_s[:64], y_train[:64], epochs=1,validation_data=(x_valid_s[:64], y_valid[:64]), verbose=0)
model.evaluate(x_test_s[:64], y_test[:64], verbose=0)

## Examples:
Train a model using a custom training loop to tackle the Fashion MNIST dataset:
- Display the epoch, iteration, mean training loss, and mean accuracy over each epoch (updated at each iteration), as well as the validation loss and accuracy at the end of each epoch.


In [5]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full.astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test.astype(np.float32) / 255.

In [6]:
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

    
def print_status_bar(step, total, loss, metrics=None):
    metrics = " - ".join([f"{m.name}: {m.result():.4f}"
                          for m in [loss] + (metrics or [])])
    end = "" if step < total else "\n"
    print(f"\r{step}/{total} - " + metrics, end=end)


def cls():
    tf.keras.backend.clear_session()
    np.random.seed(42)
    tf.random.set_seed(42)
cls()

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax"),
])

In [8]:
n_epochs = 2
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = tf.keras.losses.sparse_categorical_crossentropy
mean_loss = tf.keras.metrics.Mean()
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

In [9]:
from tqdm.notebook import trange
from collections import OrderedDict

with trange(1, n_epochs + 1, desc="All epochs") as epochs:
    for epoch in epochs:
        with trange(1, n_steps + 1, desc=f"Epoch {epoch}/{n_epochs}") as steps:
            for step in steps:
                X_batch, y_batch = random_batch(X_train, y_train)
                with tf.GradientTape() as tape:
                    y_pred = model(X_batch)
                    main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
                    loss = tf.add_n([main_loss] + model.losses)
                gradients = tape.gradient(loss, model.trainable_variables)
                optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                for variable in model.variables:
                    if variable.constraint is not None:
                        variable.assign(variable.constraint(variable))                    
                status = OrderedDict()
                mean_loss(loss)
                status["loss"] = mean_loss.result().numpy()
                for metric in metrics:
                    metric(y_batch, y_pred)
                    status[metric.name] = metric.result().numpy()
                steps.set_postfix(status)
            y_pred = model(X_valid)
            status["val_loss"] = np.mean(loss_fn(y_valid, y_pred))
            status["val_accuracy"] = np.mean(tf.keras.metrics.sparse_categorical_accuracy(
                tf.constant(y_valid, dtype=np.float32), y_pred))
            steps.set_postfix(status)
        for metric in [mean_loss] + metrics:
            metric.reset_states()

All epochs:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1/2:   0%|          | 0/1718 [00:00<?, ?it/s]

Epoch 2/2:   0%|          | 0/1718 [00:00<?, ?it/s]

- Try using a different optimizer with a different learning rate for the upper layers and the lower layers.

In [12]:
cls()

lower_layers = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(10, activation="relu"),
])

upper_layers = tf.keras.Sequential([tf.keras.layers.Dense(10, activation="softmax")])

model = tf.keras.Sequential([lower_layers, upper_layers])

lower_optimizer = tf.keras.optimizers.SGD(learning_rate=1e-4)
upper_optimizer = tf.keras.optimizers.Nadam(learning_rate=1e-3)

In [13]:
n_epochs = 2
batch_size = 32
n_steps = len(X_train) // batch_size
loss_fn = tf.keras.losses.sparse_categorical_crossentropy
mean_loss = tf.keras.metrics.Mean()
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

In [14]:
with trange(1, n_epochs + 1, desc="All epochs") as epochs:
    for epoch in epochs:
        with trange(1, n_steps + 1, desc=f"Epoch {epoch}/{n_epochs}") as steps:
            for step in steps:
                X_batch, y_batch = random_batch(X_train, y_train)
                with tf.GradientTape(persistent=True) as tape:
                    y_pred = model(X_batch)
                    main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
                    loss = tf.add_n([main_loss] + model.losses)
                for layers, optimizer in ((lower_layers, lower_optimizer),
                                          (upper_layers, upper_optimizer)):
                    gradients = tape.gradient(loss, layers.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, layers.trainable_variables))
                del tape
                for variable in model.variables:
                    if variable.constraint is not None:
                        variable.assign(variable.constraint(variable))                    
                status = OrderedDict()
                mean_loss(loss)
                status["loss"] = mean_loss.result().numpy()
                for metric in metrics:
                    metric(y_batch, y_pred)
                    status[metric.name] = metric.result().numpy()
                steps.set_postfix(status)
            y_pred = model(X_valid)
            status["val_loss"] = np.mean(loss_fn(y_valid, y_pred))
            status["val_accuracy"] = np.mean(tf.keras.metrics.sparse_categorical_accuracy(
                tf.constant(y_valid, dtype=np.float32), y_pred))
            steps.set_postfix(status)
        for metric in [mean_loss] + metrics:
            metric.reset_states()

All epochs:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1/2:   0%|          | 0/1718 [00:00<?, ?it/s]

Epoch 2/2:   0%|          | 0/1718 [00:00<?, ?it/s]